<a href="https://colab.research.google.com/github/bluebird702/study/blob/main/reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# google python sdk 설치
!pip install -q -U langchain langchain-google-genai langchain-community cache-pandas

In [2]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
REDDIT_CLIENT_ID=userdata.get('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET=userdata.get('REDDIT_CLIENT_SECRET')

In [4]:
import requests
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from langchain_community.document_loaders import RedditPostsLoader

def get_sentiment(llm, text):
  prompt = HumanMessage(content=f"Analyze the sentiment of the following text: '{text}'. Return 'positive', 'negative', or 'neutral' based on the sentiment analysis.")
  response = llm([prompt])
  sentiment = response.content.strip().lower()

  if 'positive' in sentiment:
      return 'positive'
  elif 'negative' in sentiment:
      return 'negative'
  else:
      return 'neutral'

def get_keyword(llm, text):
  prompt = HumanMessage(content=f"Extract just 5 keywords that matter from the given text: '{text}'. Return list them separated by commas.")
  response = llm([prompt])
  return response.content.strip().lower()

def process_post(llm, post):
  text = post.page_content
  sentiment = get_sentiment(llm, text)
  if sentiment == 'negative':
      keyword = get_keyword(llm, text)
  else:
      keyword = 'N/A'
  post_with_sentiment = {
      'title': post.metadata["post_title"],
      'score': post.metadata["post_score"],
      'sentiment': sentiment,
      'keyword' : keyword,
      'url': post.metadata["post_url"],
  }
  return post_with_sentiment

def process_posts(llm, posts):
  return [process_post(llm, post) for post in posts]

# 객체 생성
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY
)

# document loader
loader = RedditPostsLoader(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent="extractor by u/Master_Ocelot8179",
    categories=["new", "hot"],  # List of categories to load posts from
    mode="subreddit",
    search_queries=[
        "wallstreetbets",
    ],  # List of subreddits to load posts from
    number_posts=20,  # Default value is 10
)
documents = loader.load()
display(documents[:1])
display(process_posts(llm, documents[:5]))

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



[Document(page_content='Looks primed for a reversal- esp given recent trial advances. Anyone have any thoughts?', metadata={'post_subreddit': 'r/wallstreetbets', 'post_category': 'new', 'post_title': 'GUTS', 'post_score': 2, 'post_id': '1dbtaj8', 'post_url': 'https://www.reddit.com/r/wallstreetbets/comments/1dbtaj8/guts/', 'post_author': Redditor(name='wallstreettoday2021')})]

[{'title': 'GUTS',
  'score': 2,
  'sentiment': 'positive',
  'keyword': 'N/A',
  'url': 'https://www.reddit.com/r/wallstreetbets/comments/1dbtaj8/guts/'},
 {'title': 'CMCSA Annual Meeting Question: Missed My Share Purchase Deadline – Now It’s Free for All!',
  'score': 8,
  'sentiment': 'neutral',
  'keyword': 'N/A',
  'url': 'https://www.reddit.com/r/wallstreetbets/comments/1dbswl4/cmcsa_annual_meeting_question_missed_my_share/'},
 {'title': 'The best way to invest in humanoid robots?',
  'score': 0,
  'sentiment': 'positive',
  'keyword': 'N/A',
  'url': 'https://www.reddit.com/r/wallstreetbets/comments/1dbstcb/the_best_way_to_invest_in_humanoid_robots/'},
 {'title': 'NVDA call options through the split',
  'score': 1,
  'sentiment': 'negative',
  'keyword': 'nvda, call options, split, off hours effect, stock going down',
  'url': 'https://i.redd.it/yg09e030hj5d1.jpeg'},
 {'title': 'Which broker offers compound options / split-fee options ',
  'score': 0,
  'sentiment': 'neutral',
 